<a href="https://colab.research.google.com/github/lkjhgfdbf3/Identification-of-dog-eye-diseases/blob/main/MobileNet_fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -qq "/content/drive/MyDrive/01.데이터.zip"

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders

In [ ]:
splitfolders.ratio('/content/01.데이터', output="output", seed=1337, ratio=(0.7, 0.2, 0.1))

In [ ]:
from tensorflow.keras import layers
img_input = layers.Input(shape=(224*224*3))

In [ ]:
import numpy as np

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Model

mobilenet = MobileNet(weights='imagenet', include_top=False)

In [ ]:
import os

base_dir = '/content/output'

train_dir = os.path.join(base_dir, 'train')
train_normal_dir = os.path.join(train_dir, '무')
train_disease_dir = os.path.join(train_dir, '유')

validation_dir = os.path.join(base_dir, 'val')
validation_normal_dir = os.path.join(validation_dir, '무')
validation_disease_dir = os.path.join(validation_dir, '유')

In [ ]:
print('total training normal images:', len(os.listdir(train_normal_dir)))
print('total training disease images:', len(os.listdir(train_disease_dir)))
print('total validation normal images:', len(os.listdir(validation_normal_dir)))
print('total validation disease images:', len(os.listdir(validation_disease_dir)))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(224, 224),
        batch_size=20,
        #classes=['nor','dis'],
        shuffle=True,
        class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=20,
        #classes=['nor','dis'],
        shuffle=True,
        class_mode='binary')

In [ ]:
import matplotlib.pylab as plt

#class_labels = ['nor', 'dis']
batch = next(train_generator)
images, labels = batch[0], batch[1] # 0번 이미지데이터 1번 레이블
print(labels[:5])
plt.figure(figsize=(16,8))
for i in range(5) :
    ax = plt.subplot(4,8,i+1)
    plt.imshow(images[i])
    #plt.title(class_labels[labels[i].astype(np.int)])
    plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
# for layer in mobilenet.layers:
#     layer.trainable = False

In [ ]:
x = mobilenet.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(mobilenet.input, predictions)

In [ ]:
mobilenet.trainable = True
print("Number of layers in the base model: ", len(mobilenet.layers))

# 해당 층 이후부터 미세 조정
fine_tune_at = 50

# `fine_tune_at` 층 이전의 모든 층을 고정
for layer in mobilenet.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:
from tensorflow import keras

adam = keras.optimizers.Adam(learning_rate=0.00001)
checkpoint_filepath = "FMbest.hdf5"
model.compile(optimizer=adam, loss="binary_crossentropy", metrics=["accuracy"])

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=5, verbose=1)

save_best = keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss',
                                            verbose=1, save_best_only=True, save_weights_only=True, 
                                            mode='auto',save_freq='epoch',options=None)

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=27,  
      epochs=50,
      validation_data=validation_generator,
      validation_steps=10,
      callbacks=[early_stop, save_best],
      verbose=1)

In [ ]:
plt.plot(history.history['accuracy'],'b-', label='accuracy')
plt.plot(history.history['val_accuracy'],'r--', label='val_accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'],'b-', label='loss')
plt.plot(history.history['val_loss'],'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
# import os
# import shutil
# import time

# def read_all_file(path):
#     output = os.listdir(path)
#     file_list = []

#     for i in output:
#         if os.path.isdir(path+"/"+i):
#             file_list.extend(read_all_file(path+"/"+i))
#         elif os.path.isfile(path+"/"+i):
#             file_list.append(path+"/"+i)

#     return file_list

# def copy_all_file(file_list, new_path):
#     for src_path in file_list:
#         file = src_path.split("/")[-1]
#         shutil.copyfile(src_path, new_path+"/"+file)
#         print("파일 {} 작업 완료".format(file)) # 작업한 파일명 출력
        
        
# start_time = time.time() # 작업 시작 시간 

# src_path = "/content/output/test/유" # 기존 폴더 경로
# new_path = "/content/output/test_2/test_folder" # 옮길 폴더 경로

# file_list = read_all_file(src_path)
# copy_all_file(file_list, new_path)

# print("=" * 40)
# print("러닝 타임 : {}".format(time.time() - start_time)) # 총 소요시간 계산

In [ ]:
# test_dir1 = os.path.join(base_dir, 'test_2')

# test_datagen1 = ImageDataGenerator(rescale=1./255)

# test_generator1 = test_datagen1.flow_from_directory(
#         directory=test_dir1,
#         target_size=(224, 224),
#         batch_size=1,
#         classes=['.'],
#         shuffle=False,
#         class_mode='binary')

In [ ]:
# test_dir = os.path.join(base_dir, 'test')

# test_datagen = ImageDataGenerator(rescale=1./255)

# test_generator = test_datagen.flow_from_directory(
#         test_dir,
#         target_size=(224, 224),
#         batch_size=20,
#         #classes=['nor','dis'],
#         shuffle=False,
#         class_mode='binary')

In [ ]:
# model.evaluate(test_generator1)

In [ ]:
# # Retrieve a batch of images from the test set
# import tensorflow as tf

# image_batch, label_batch = test_generator1.next()
# predictions = model.predict_on_batch(image_batch).flatten()

# # Apply a sigmoid since our model returns logits
# predictions = tf.nn.sigmoid(predictions)
# predictions = tf.where(predictions < 0.5, 0, 1)

# print('Predictions:\n', predictions.numpy())
# print('Labels:\n', label_batch)

# plt.figure(figsize=(10, 10))
# for i in range(9):
#   ax = plt.subplot(3, 3, i + 1)
#   plt.imshow(image_batch[i])

#   plt.axis("off")

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions
import tensorflow as tf

img_path = '/content/t.JPG'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
print(preds)
predictions = tf.nn.sigmoid(preds)
preds = tf.where(preds < 0.5, 0, 1)
print(preds)
plt.imshow(img)